<a href="https://colab.research.google.com/github/ciepielajan/Multi-Class-Classification-NLP/blob/main/Detecting_intentions_CNN_KerasClassifier_i_RandomizedSearchCV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

KerasClassifier i Randomized Search Cross Validation

#### Pobranie danych

In [ ]:
# https://drive.google.com/file/d/1fI6EXyD9TMTC1jzdu206ljXOGNjdHprq/view?usp=sharing
!gdown --id "1fI6EXyD9TMTC1jzdu206ljXOGNjdHprq"

Downloading...
From: https://drive.google.com/uc?id=1fI6EXyD9TMTC1jzdu206ljXOGNjdHprq
To: /content/user_intent.zip
100% 271k/271k [00:00<00:00, 37.1MB/s]


In [ ]:
!unzip "user_intent.zip"

Archive:  user_intent.zip
  inflating: train.csv               
  inflating: __MACOSX/._train.csv    
  inflating: validation.csv          


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

data_set_train = pd.read_csv("train.csv")
data_set_valid = pd.read_csv("validation.csv")

print(data_set_train.shape)
print(data_set_valid.shape)

print(data_set_train.columns)
print(data_set_valid.columns)

(13784, 2)
(700, 2)
Index(['text', 'label'], dtype='object')
Index(['text', 'label'], dtype='object')


#### Podstawowy process text

In [ ]:
import re
def process_text(sentence):
    sentence = re.sub('[A-Za-z0-9]+@[a-zA-z].[a-zA-Z]+', '', sentence)  # maile
    sentence = re.sub('(http[s]*:[/][/])[a-zA-Z0-9]+', '', sentence)  # linki
    sentence = re.sub(r"<[^>]+>", " ", sentence) # remove html tag
    sentence = re.sub(r"[^a-zA-Z\s]", "", sentence)  # remove punctuations and numbers   
    sentence = re.sub(r"\b[A-Za-z]{1}\b", "", sentence)  # remove single characters

    sentence = re.sub("^\s+|\s+$", "", sentence, flags=re.UNICODE) # Remove spaces both in the BEGINNING and in the END of a string:
    sentence = " ".join(re.split("\s+", sentence, flags=re.UNICODE))  # Remove ONLY DUPLICATE spaces:
    sentence = sentence.lower()
    
    return sentence

In [ ]:
data_set_train["clean_text"] = data_set_train["text"].apply(lambda x: process_text(x))

#### `LabelEncoder` oraz `to_categorical`

In [ ]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical 


labelencoder = LabelEncoder()
data_set_train["labelencoder"] = labelencoder.fit_transform(data_set_train["label"])

dummy_y = to_categorical(data_set_train["labelencoder"], dtype ="float32")

data_set_train[["clean_text","label","labelencoder"]].head(5)

,clean_text,label,labelencoder
0,find cinema nearest for films,SearchScreeningEvent,6
1,give the current series two stars,RateBook,4
2,find the good girl at movie house,SearchScreeningEvent,6
3,please make reservations for three at kosher t...,BookRestaurant,1
4,what is the forecast for here one second from now,GetWeather,2


In [ ]:
dummy_y.shape

(13784, 7)

In [ ]:
id_intention = 6
print("Sprawdzenie poprawności LabelEncoder i to_categorical \n")
print("Label - ", data_set_train["label"].iloc[id_intention])
print("LabelEncoder - ", data_set_train["labelencoder"].iloc[id_intention])
print()
print("to_categorical - ", dummy_y[id_intention])
print()
print("return to LabelEncoder - ",np.argmax(dummy_y[id_intention], axis=-1))
print("return to Label - ",labelencoder.inverse_transform([np.argmax(dummy_y[id_intention], axis=-1)]))

Sprawdzenie poprawności LabelEncoder i to_categorical 

Label -  BookRestaurant
LabelEncoder -  1

to_categorical -  [0. 1. 0. 0. 0. 0. 0.]

return to LabelEncoder -  1
return to Label -  ['BookRestaurant']


#### Zdefiniowanie X i y

In [ ]:
X = data_set_train["clean_text"]
y = dummy_y

In [ ]:
X.shape, y.shape

((13784,), (13784, 7))


#### `KerasClassifier` i `Randomized Search Cross Validation`

`Randomized Search Cross Validation`

Wykorzystuje to losowy zestaw hiperparametrów. Przydatne, gdy istnieje wiele hiperparametrów, więc przestrzeń wyszukiwania jest duża. Może być używany, jeśli masz wcześniejsze przekonanie o tym, jakie powinny być hiperparametry. 

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

In [ ]:
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
from keras.layers import Dense, Activation, Embedding, Flatten, GlobalMaxPooling1D, Dropout, Conv1D

In [ ]:
def create_model(num_filters, kernel_size, vocab_size, embedding_dim, maxlen):
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_dim, input_length=maxlen))
    model.add(Conv1D(num_filters, kernel_size, activation='relu'))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(7, activation="softmax")) 
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    return model

In [ ]:


# df z wynikami wszystkich RandomizedSearchCV
wyniki = pd.DataFrame()
wyniki

from keras.wrappers.scikit_learn import KerasClassifier  
from sklearn.model_selection import RandomizedSearchCV


# Main settings
epochs = 20
embedding_dim = 100
maxlen = 13

# Run grid search for each source (yelp, amazon, imdb)
# for source, frame in df.groupby('source'):
# print('Running grid search for data set :', source)
sentences = data_set_train["clean_text"].values  
y = dummy_y

# Train-test split
sentences_train, sentences_test, y_train, y_test = train_test_split(sentences, y, test_size=0.25, random_state=1000)

# Tokenize words
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(sentences_train)
X_train = tokenizer.texts_to_sequences(sentences_train)
X_test = tokenizer.texts_to_sequences(sentences_test)

# Adding 1 because of reserved 0 index
vocab_size = len(tokenizer.word_index) + 1

# Pad sequences with zeros
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

# Parameter grid for grid search
param_grid = dict(num_filters=[32, 64, 128],
                  kernel_size=[3, 5, 7],
                  vocab_size=[vocab_size],
                  embedding_dim=[100,200],
                  maxlen=[maxlen],
                  epochs = [20],
                  batch_size=[16,32,64] )

model = KerasClassifier(build_fn=create_model,
                        verbose=False)


grid = RandomizedSearchCV(estimator=model, param_distributions=param_grid, cv=4, verbose=10, n_iter=5)  #n_liter odpowiada za ilość radomize serach
# grid_result = grid.fit(X_train, y_train)

from keras.callbacks import EarlyStopping
# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
grid_result = grid.fit(X_train, y_train, validation_split=0.2, callbacks=[early_stopping])  #validation_split=0.2

# Evaluate testing set
test_accuracy = grid.score(X_test, y_test)
grid_result.best_params_.update({"best_score_":grid_result.best_score_})
wyniki = wyniki.append(grid_result.best_params_, ignore_index=True)

Fitting 4 folds for each of 5 candidates, totalling 20 fits
[CV] vocab_size=9462, num_filters=64, maxlen=13, kernel_size=5, epochs=20, embedding_dim=100, batch_size=32 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  vocab_size=9462, num_filters=64, maxlen=13, kernel_size=5, epochs=20, embedding_dim=100, batch_size=32, score=0.978, total=  32.6s
[CV] vocab_size=9462, num_filters=64, maxlen=13, kernel_size=5, epochs=20, embedding_dim=100, batch_size=32 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   32.6s remaining:    0.0s


[CV]  vocab_size=9462, num_filters=64, maxlen=13, kernel_size=5, epochs=20, embedding_dim=100, batch_size=32, score=0.983, total=  31.4s
[CV] vocab_size=9462, num_filters=64, maxlen=13, kernel_size=5, epochs=20, embedding_dim=100, batch_size=32 


...

[CV]  vocab_size=9462, num_filters=128, maxlen=13, kernel_size=7, epochs=20, embedding_dim=100, batch_size=16, score=0.978, total=  40.9s
[CV] vocab_size=9462, num_filters=128, maxlen=13, kernel_size=7, epochs=20, embedding_dim=100, batch_size=16 


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  4.7min remaining:    0.0s


[CV]  vocab_size=9462, num_filters=128, maxlen=13, kernel_size=7, epochs=20, embedding_dim=100, batch_size=16, score=0.986, total=  52.8s
[CV] vocab_size=9462, num_filters=128, maxlen=13, kernel_size=7, epochs=20, embedding_dim=100, batch_size=16 
[CV]  vocab_size=9462, num_filters=128, maxlen=13, kernel_size=7, epochs=20, embedding_dim=100, batch_size=16, score=0.982, total=  42.5s
[CV] vocab_size=9462, num_filters=128, maxlen=13, kernel_size=7, epochs=20, embedding_dim=100, batch_size=16 
[CV]  vocab_size=9462, num_filters=128, maxlen=13, kernel_size=7, epochs=20, embedding_dim=100, batch_size=16, score=0.985, total=  55.9s
[CV] vocab_size=9462, num_filters=128, maxlen=13, kernel_size=3, epochs=20, embedding_dim=200, batch_size=32 
[CV]  vocab_size=9462, num_filters=128, maxlen=13, kernel_size=3, epochs=20, embedding_dim=200, batch_size=32, score=0.978, total=  50.4s
[CV] vocab_size=9462, num_filters=128, maxlen=13, kernel_size=3, epochs=20, embedding_dim=200, batch_size=32 
[CV]  vo

[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed: 13.9min finished


CPU times: user 25min 23s, sys: 1min 18s, total: 26min 41s
Wall time: 14min 59s


In [ ]:
wyniki

,batch_size,best_score_,embedding_dim,epochs,kernel_size,maxlen,num_filters,vocab_size
0,32.0,0.982879,200.0,20.0,3.0,13.0,128.0,9462.0


In [ ]:
print("#### RandomizedSearchCV")
print(wyniki.to_markdown())

#### RandomizedSearchCV
|    |   batch_size |   best_score_ |   embedding_dim |   epochs |   kernel_size |   maxlen |   num_filters |   vocab_size |
|---:|-------------:|--------------:|----------------:|---------:|--------------:|---------:|--------------:|-------------:|
|  0 |           32 |      0.982879 |             200 |       20 |             3 |       13 |           128 |         9462 |


#### Predykcja na zbiorze validacyjnym